In [1]:
from pathlib import Path
import pandas as pd
from pymongo import MongoClient

In [2]:
host = '203.65.72.120'
client = MongoClient(host=host)

In [3]:
client.list_database_names()

['Listeria_monocytogenes',
 'Mycobacterium_tuberculosis_variant_bovis',
 'Neisseria_meningitidis',
 'Salmonella_enterica',
 'Vibrio_cholerae',
 'admin',
 'config',
 'local']

In [4]:
database = client['Listeria_monocytogenes']

In [5]:
database.list_collection_names()

['Description', 'Sample_attribute']

In [6]:
collettion = database['Sample_attribute']

In [12]:
document_filter = {}

In [13]:
cursor = collettion.find(document_filter, {'profile': 0, '_id': 0})

In [14]:
metadata = pd.DataFrame(cursor)
metadata

,NCBIAccession,BioSample,Country,Year,Serovar,cgMLST,ST,ResistanceGene,Identifier,Host
0,SRR1553882,SAMN02991377,USA,2014,4,133,382,"fosX, lin",MOD1_LS985,NaN
1,SRR1553851,SAMN02991392,USA,2014,4,219,382,"fosX, lin",MOD1_LS1008,NaN
2,SRR1553739,SAMN02991375,USA,2014,4,219,382,"fosX, lin",MOD1_LS982,NaN
3,SRR1553866,SAMN02990447,USA,2014,4,219,382,"fosX, lin",MOD1_LS984,NaN
4,SRR1553750,SAMN02990450,USA,2014,4,219,382,"fosX, lin",MOD1_LS988,NaN
...,...,...,...,...,...,...,...,...,...,...
14932,SRR7912066,SAMN10141040,Taiwan,2014,1/2b,81,87,"blaCTX-M, blaTEM, fosX, lin",R14.0396,Homo sapiens
14933,SRR7912065,SAMN10141039,Taiwan,2014,1/2b,1482,87,"fosX, lin",R14.0372,Homo sapiens
14934,SRR7912236,SAMN10141037,Taiwan,2014,1/2a,677,155,"fosX, lin",R14.0254,Homo sapiens
14935,SRR7912193,SAMN10141036,Taiwan,2014,1/2b,1567,5,"fosX, lin",R14.0214,Homo sapiens


In [15]:
metadata = metadata[metadata['Year'].apply(lambda x: pd.isna(x) or x.isdigit())]

In [16]:
metadata.to_csv('/home/chen1i6c04/data/Lm_metadata.csv', index=False)

In [19]:
cursor = collettion.find({'NCBIAccession': {'$in': metadata['NCBIAccession'].to_list()}}, {'profile': 1, '_id': 0, 'NCBIAccession': 1})

In [20]:
outpath = Path('/home/chen1i6c04/data')
for document in cursor:
    s = pd.Series(document['profile'])
    s.index.name, s.name = 'locus_id', 'allele_id'
    outfile = outpath/(document['NCBIAccession'] + '.tsv')
    s.to_csv(outfile, sep='\t')

In [10]:
import json

In [22]:
locus_tags = []
with open('/media/GenomicResearch/Issue/Benga/Listeria_monocytogenes.faa') as handle:
    for line in handle:
        if line.startswith('>'):
            locus_tags.append(line.strip().split()[0].replace('>', ''))

In [23]:
description = dict()
description['fields'] = metadata.columns.to_list()
description['core_genome'] = locus_tags

In [26]:
with open('/home/chen1i6c04/data/db.json', 'w') as f:
    json.dump(description, f)